#### TMA4205 Numerical Linear Algebra
# Project Part 2
## Introduction

In this part we want to find a low rank continuous approximation to a continuous matrix valued function.

In [ ]:
%load_ext autoreload
%autoreload 2

# imports and useful functions
import matplotlib.pyplot as plt

# line cyclers adapted to colourblind people
from cycler import cycler

line_cycler = (cycler(color=["#E69F00", "#56B4E9", "#009E73", "#0072B2", "#D55E00", "#CC79A7", "#F0E442"]) +
               cycler(linestyle=["-", "--", "-.", ":", "-", "--", "-."]))
plt.rc("axes", prop_cycle=line_cycler)
plt.rc('axes', axisbelow=True)

In [ ]:
"""First define some useful functions for plotting and testing"""
from linalg.plotting import *

## Exercise 1:
...